Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [65]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
import random
from sklearn.model_selection import train_test_split

In [66]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [67]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [68]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [69]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
# network.add(layer(ActFunc.relu, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [70]:
X_demo = X_train.head()
X_demo

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
571,228.0,342.1,0.0,185.7,0.0,955.8,674.3,28
623,307.0,0.0,0.0,193.0,0.0,968.0,812.0,3
740,297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7
750,500.0,0.0,0.0,200.0,0.0,1125.0,613.0,28
262,212.6,0.0,100.4,159.4,10.4,1003.8,903.8,56


In [71]:
y_demo = y_train.head()

In [72]:
pos = np.random.rand(31)

weights = pos[0:27]
weights_arr = []
j = 0
for i in range(len(network.layers)):
    weight_mat = np.reshape(weights[j:j + (network.layers[i].numOfNodes * 8)], (network.layers[i].numOfNodes, -1))
    weights_arr.append(weight_mat)
    j = j + (network.layers[i].numOfNodes * 8)
    input = network.layers[i].numOfNodes


bias = pos[27:31] # Total num of nodes in the NN
bias_arr = []
j = 0
for i in range(len(network.layers)):
    bias_arr.append(bias[j:j + network.layers[i].numOfNodes])
    j = j + network.layers[i].numOfNodes

weights_arr

[array([[0.76575868, 0.21655623, 0.19299049, 0.55791436, 0.99688631,
         0.68208063, 0.5236384 , 0.76019424],
        [0.56890031, 0.46995342, 0.3943193 , 0.67726879, 0.9795098 ,
         0.15424205, 0.74708649, 0.74482414],
        [0.59258401, 0.7862399 , 0.56181866, 0.65082354, 0.26913467,
         0.16440441, 0.83366839, 0.20638092]]),
 array([[0.50428723, 0.42544738, 0.634402  ]])]

In [73]:
yhat = X_demo.apply(network.forwardCalculation, args = (weights_arr, bias_arr), axis = 1)
network.sseCalculation(yhat, y_demo)

18117965.18336774

In [74]:
class Particle:

    def __init__(self, pos, velo, alpha, beta, gamma, delta):
        self.pos = pos
        self.velo = velo
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.prevBest = pos # Not sure should put pos or None here
        self.informant = []

    def changeVelo(self, xloc, xbest, infbest, globbest):
        for i in range(len(self.velo)):
            # Generate rand num from beta, gamma and delta
            b = np.random.rand() * self.beta
            c = np.random.rand() * self.gamma
            d = np.random.rand() * self.delta
            self.velo[i] = self.alpha * self.velo[i] + b * (xbest[i] - xloc[i]) + c * (infbest[i] - xloc[i]) + d * (globbest[i] - xloc[i])

In [75]:
network.layers[0].numOfNodes

3

In [90]:
# PSO class to train weight
# The inputLen is the num of attribute of the cement: 8

class PSO:
    

    def __init__(self, X, y, network, swarmsize, alpha, beta, gamma, delta, epsilon):
        self.X = X
        self.y = y
        self.swarmsize = swarmsize
        self.network = network
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon
        self.particles = None

    def particleDim(self):
        dim = 0
        inputLen = len(X.columns)
        for i in range(len(self.network.layers)):
            dim = dim + ((self.network.layers[i].numOfNodes * inputLen) + self.network.layers[i].numOfNodes) # plus 1 for bias
            inputLen = self.network.layers[i].numOfNodes
        return dim
    
    # Array of random particles
    def randParticle(self):
        self.particles = []
        for i in range(self.swarmsize):
            #np.random.seed(0) # fix the random for easier checking
            pos = np.random.rand(self.particleDim())
            velo = np.random.rand(self.particleDim())
            self.particles.append(Particle(pos, velo, self.alpha, self.beta, self.gamma, self.delta)) # Try to add rand particle here
        return self.particles
    
    # Randomly assign informants to particle
    def randAssignInformant(self, numOfInformant):
        for particle in self.particles:
            other_particles = self.particles.copy()
            other_particles.remove(particle)
            for i in range(numOfInformant):
                particle.informant = random.sample(other_particles, numOfInformant)

    # Return the fittest location of all particles given
    def fittestLoc(self, someParticles):
        fittest_pos = None
        fittest_sse = None
        for p in someParticles: # supposed to be particle instead of list
            sse = self.assessFitness(p.pos)
            if fittest_sse == None or sse < fittest_sse:
                fittest_sse = sse
                fittest_pos = p.pos
        return fittest_pos



    # return the nice weight and bias matrix
    def assessFitness_helper(self, pos):
        # Put the weights into a nice matrix
        weights_arr = []
        inputLen = len(X.columns)
        temp_inputLen = inputLen
        j = 0
        for i in range(len(network.layers)):
            weight_mat = np.reshape(pos[j:j + (network.layers[i].numOfNodes * temp_inputLen)], (network.layers[i].numOfNodes, -1))
            weights_arr.append(weight_mat)
            j = j + (network.layers[i].numOfNodes * temp_inputLen)
            temp_inputLen = network.layers[i].numOfNodes

        # Put the bias into a nice matrix
        bias_arr = []
        for i in range(len(network.layers)):
            bias_arr.append(pos[j:j + network.layers[i].numOfNodes])
            j = j + network.layers[i].numOfNodes

        return weights_arr, bias_arr

    # Check fitness of the particle
    # Also saves new best to the particle
    def assessFitness(self, pos):
        # Calculate sse of current pos
        weights_arr, bias_arr = self.assessFitness_helper(pos)
        
        # Calculate the pred y
        yhat = X.apply(network.forwardCalculation, args = (weights_arr, bias_arr), axis = 1)
        sse = network.sseCalculation(yhat, y)

        return sse
    
    # (Additional) Calculate sse of prev best pos and compare
    def isNewBest(self, sse, particle):
        
        prevBest = particle.prevBest
        sse_prev = self.assessFitness(prevBest)

        if sse < sse_prev:
            particle.prevBest = pos

    def optimise(self):
        inputLen = len(X.columns)
        # Initialize rand particle and global best
        self.randParticle()
        self.randAssignInformant(1)
        best = self.particles[0]
        best_sse = None

        # There should be another for loop here covering all the steps until we reach an optimal best
        #while (self.assessFitness(best.pos) >= 1000):
        for i in range(10):
            # Access fitness of each particle (set of weights)
            for particle in self.particles:
                sse = self.assessFitness(particle.pos)
                self.isNewBest(sse, particle)
                if best_sse == None or sse < best_sse:
                    best = particle
                    best_sse = sse
                
            # Gather information
            for particle in self.particles:
                x_star = particle.prevBest
                x_plus = self.fittestLoc(particle.informant)
                x_prime = self.fittestLoc(self.particles)

                # Do the change velo
                particle.changeVelo(particle.pos, x_star, x_plus, x_prime)
                    
                # Do the change pos
                particle.pos = particle.pos + (self.epsilon * particle.velo)


    

In [ ]:
swarmsize = 10
alpha = 0.5
beta = 1
gamma = 1
delta = 2
epsilon = 0.3

pso = PSO(X_demo, y_demo, network, swarmsize, alpha, beta, gamma, delta, epsilon)
pso.optimise()



In [78]:
print(pso.particles)
for i in range(10):
    print(pso.particles[i].informant)


[<__main__.Particle object at 0x000002D5B8AAA390>, <__main__.Particle object at 0x000002D5B8AA8E60>, <__main__.Particle object at 0x000002D5B8AA8DA0>, <__main__.Particle object at 0x000002D5B8AABC20>, <__main__.Particle object at 0x000002D5B8AA9E50>, <__main__.Particle object at 0x000002D5B8AAA5A0>, <__main__.Particle object at 0x000002D5B8AAB0B0>, <__main__.Particle object at 0x000002D5B9696450>, <__main__.Particle object at 0x000002D5B8AABDA0>, <__main__.Particle object at 0x000002D5B8AA9E80>]


In [79]:
pso.particles[i].informant[0].pos

array([0.20740845, 0.5652148 , 0.09802142, 0.58205964, 0.76560153,
       0.31056209, 0.47951266, 0.09715622, 0.99491792, 0.51481119,
       0.64691362, 0.30714249, 0.99990525, 0.37501182, 0.15612431,
       0.35211383, 0.84577009, 0.26908592, 0.49353855, 0.93897218,
       0.366603  , 0.10789574, 0.15883455, 0.65419198, 0.70563723,
       0.68542392, 0.5456254 , 0.07092857, 0.61598155, 0.25093567,
       0.32690304])